### **Library Installation** :
Installs required libraries like Scikit-learn, Gradio, and Sentence Transformers, and downloads the Spacy English model. (Time: 30 sec)

In [30]:
!pip install -q pandas numpy scikit-learn gradio rapidfuzz pyspellchecker sentence-transformers spacy joblib imbalanced-learn
!python -m spacy download en_core_web_sm > /dev/null 2>&1 || true


### **Imports and Configuration** :
Imports libraries, creates model directories, and sets global constants and random seeds for reproducibility. (Time: 20 sec)

In [31]:
import os
import re
import time
import json
import joblib
import requests
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.sparse import hstack
from rapidfuzz import process, fuzz

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

from spellchecker import SpellChecker
import spacy
from sentence_transformers import SentenceTransformer, util
import gradio as gr

MODEL_DIR = Path("/content/models")
MODEL_DIR.mkdir(parents=True, exist_ok=True)

EMBED_MODEL = "multi-qa-mpnet-base-dot-v1"
SEMANTIC_THRESHOLD = 0.66
FUZZY_THRESHOLD = 72
NB_PROBA_THRESHOLD = 0.60

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


### **Data Loading** :
Downloads the JSON dataset from GitHub, parses it into a Pandas DataFrame, and removes duplicate questions. (Time: 1-5 secs)

In [32]:
import requests
import pandas as pd

# first dataset
URL1 = "https://raw.githubusercontent.com/saghiralich/AI-project-Dataset/main/DatasetAIP.json"
# New dataset (551-980)
URL2 = "https://raw.githubusercontent.com/saghiralich/AI-project-Dataset/main/DatasetAIP2.json"

def load_dataset_safe(url):
    resp = requests.get(url)
    resp.raise_for_status()
    data_json = resp.json()
    if isinstance(data_json, dict) and 'data' in data_json:
        records = data_json['data']
    elif isinstance(data_json, list):
        if len(data_json) >= 1 and isinstance(data_json[0], dict) and 'data' in data_json[0]:
            records = data_json[0]['data']
        else:
            records = data_json
    else:
        raise ValueError("Unexpected JSON structure")

    normalized = []
    for item in records:
        if not isinstance(item, dict):
            continue
        row = {
            'id': item.get('id', None),
            'question': str(item.get('question','')).strip(),
            'answer': str(item.get('answer','')).strip(),
            'category': item.get('category', item.get('intent','unknown')),
            'intent': item.get('intent',''),
            'entities': item.get('entities') if isinstance(item.get('entities'), dict) else {}
        }
        normalized.append(row)
    return pd.DataFrame(normalized)

_df1 = load_dataset_safe(URL1)
_df2 = load_dataset_safe(URL2)
df = pd.concat([_df1, _df2], ignore_index=True)
df = df.drop_duplicates(subset=['question'])

print("Total combined rows:", len(df))
df.head(3)


Total combined rows: 951


,id,question,answer,category,intent,entities
0,1,What is the fee for BS Computer Science?,The fee for BS Computer Science is approximate...,fee_structure,program_fee_inquiry,"{'program': 'BS Computer Science', 'fee_type':..."
1,2,How much is the MBA program?,"The MBA program fee is around Rs. 115,000 per ...",fee_structure,program_fee_inquiry,"{'program': 'MBA', 'fee_type': 'tuition'}"
2,3,When is the fee payment deadline?,The fee payment deadline is typically within t...,fee_structure,deadline_inquiry,"{'deadline_type': 'fee_payment', 'penalty': 'l..."


### **Text Preprocessing** :
Cleans text by fixing spelling, expanding abbreviations (e.g., "cs" to "computer science"), and lemmatizing words. (Time: ~1.5 mins)

In [33]:
spell = SpellChecker()
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])
SYNONYMS = {
    "ai": "artificial intelligence",
    "a.i": "artificial intelligence",
    "cs": "computer science",
    "bs": "bachelor of science",
    "ms": "master of science",
    "uni": "university",
    "dept": "department",
    "admsn": "admission",
}

RE_NON_ALNUM = re.compile(r"[^a-z0-9\s]")
RE_MULTI_SP = re.compile(r"\s+")
_spell_cache = {}

def spell_correct_token(t):
    if len(t) <= 2:
        return t
    if t in _spell_cache:
        return _spell_cache[t]
    try:
        c = spell.correction(t)
    except Exception:
        c = t
    _spell_cache[t] = c if c else t
    return _spell_cache[t]

def normalize_fast(text):
    if pd.isna(text):
        return ''
    text = str(text).lower()
    text = RE_NON_ALNUM.sub(' ', text)
    text = RE_MULTI_SP.sub(' ', text).strip()
    tokens = text.split()
    tokens = [SYNONYMS.get(tok, tok) for tok in tokens]
    tokens = [spell_correct_token(tok) for tok in tokens]
    joined = ' '.join(tokens)
    doc = nlp(joined)
    lemmas = [tok.lemma_ for tok in doc if not tok.is_space]
    out = ' '.join(lemmas)
    out = RE_MULTI_SP.sub(' ', out).strip()
    return out

start = time.time()
df['question_proc'] = df['question'].apply(normalize_fast)
df['answer_proc'] = df['answer'].apply(lambda x: normalize_fast(str(x)))
print("Preprocessing time (s):", round(time.time() - start,2))
df[['question','question_proc']].head(5)


Preprocessing time (s): 81.91


,question,question_proc
0,What is the fee for BS Computer Science?,what be the fee for bachelor of science comput...
1,How much is the MBA program?,how much be the ma program
2,When is the fee payment deadline?,when be the fee payment deadline
3,Can I pay my fee in installments?,can I pay my fee in installment
4,Are there any additional charges?,be there any additional charge


### **Feature Extraction (TF-IDF)** :
Convert text into numerical features using Word and Character TF-IDF vectorizers and saves them to disk. (Time: 10 secs)

In [34]:
word_vect = TfidfVectorizer(analyzer='word', ngram_range=(1,2), max_df=0.95, min_df=1, sublinear_tf=True)
char_vect = TfidfVectorizer(analyzer='char', ngram_range=(3,5), max_df=0.95, min_df=1, sublinear_tf=True)

Xw = word_vect.fit_transform(df['question_proc'])
Xc = char_vect.fit_transform(df['question_proc'])
X = hstack([Xw, Xc])

le = LabelEncoder()
y = le.fit_transform(df['category'])

print("Feature shapes:", Xw.shape, Xc.shape, X.shape)
joblib.dump(word_vect, MODEL_DIR / "word_tfidf.joblib")
joblib.dump(char_vect, MODEL_DIR / "char_tfidf.joblib")
joblib.dump(le, MODEL_DIR / "label_encoder.joblib")


Feature shapes: (951, 3247) (951, 15474) (951, 18721)


['/content/models/label_encoder.joblib']

### **Model Training (Classifiers)** :
Balances data with SMOTE, trains a Voting Classifier (Naive Bayes, LR, SVM), and saves the trained models. (Time: 30 secs)

In [35]:
from imblearn.over_sampling import SMOTE

category_counts = df['category'].value_counts()
singleton_categories = category_counts[category_counts == 1].index.tolist()
df_filtered = df[~df['category'].isin(singleton_categories)].copy() if singleton_categories else df.copy()

le = LabelEncoder()
y_filtered = le.fit_transform(df_filtered['category'])

Xw_filtered = word_vect.transform(df_filtered['question_proc'])
Xc_filtered = char_vect.transform(df_filtered['question_proc'])
X_filtered = hstack([Xw_filtered, Xc_filtered])

X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X_filtered, y_filtered, df_filtered.index,
    test_size=0.15, random_state=RANDOM_STATE, stratify=y_filtered
)
print("Train/test sizes:", X_train.shape[0], X_test.shape[0])

smote = SMOTE(random_state=RANDOM_STATE)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print("Resampled training set shape:", X_train_res.shape, y_train_res.shape)

nb = MultinomialNB(alpha=0.5)
nb.fit(X_train_res, y_train_res)

cnb = ComplementNB()
cnb.fit(X_train_res, y_train_res)

lr = LogisticRegression(max_iter=2000, n_jobs=-1)
lr.fit(X_train_res, y_train_res)

svc = LinearSVC(max_iter=2000)
svc_cal = CalibratedClassifierCV(svc, cv=3)
svc_cal.fit(X_train_res, y_train_res)

voting = VotingClassifier(
    estimators=[('nb', nb), ('cnb', cnb), ('lr', lr)],
    voting='soft'
)
voting.fit(X_train_res, y_train_res)
for name, model in [('NB', nb), ('CNB', cnb), ('LR', lr), ('SVC_cal', svc_cal), ('Voting', voting)]:
    acc = accuracy_score(y_test, model.predict(X_test))
    print(f"{name} Accuracy: {acc:.4f}")
y_pred_voting = voting.predict(X_test)
report = classification_report(y_test, y_pred_voting, target_names=le.classes_, output_dict=True)
overall_acc = report['accuracy']
print(f"\nVoting classifier simplified report: Accuracy = {overall_acc:.4f}")

joblib.dump(nb, MODEL_DIR / "nb_model.joblib")
joblib.dump(voting, MODEL_DIR / "voting_model.joblib")
joblib.dump(svc_cal, MODEL_DIR / "svc_calibrated.joblib")

Train/test sizes: 806 143
Resampled training set shape: (3213, 18721) (3213,)
NB Accuracy: 0.6643
CNB Accuracy: 0.6224
LR Accuracy: 0.7552
SVC_cal Accuracy: 0.7133
Voting Accuracy: 0.6713

Voting classifier simplified report: Accuracy = 0.6713


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


['/content/models/svc_calibrated.joblib']

### **Semantic Embeddings** :
Generates deep learning embeddings for all questions to allow the bot to understand semantic intent/meaning. (Time: 1-2 mins)

In [36]:
print("Loading embedder (this may take a bit)...")
embedder = SentenceTransformer(EMBED_MODEL)
questions = df['question_proc'].tolist()
question_embeddings = embedder.encode(questions, convert_to_tensor=True, show_progress_bar=True, batch_size=64)
joblib.dump(question_embeddings, MODEL_DIR / "question_embeddings.joblib")
print("Embeddings computed:", question_embeddings.shape)


Loading embedder (this may take a bit)...


Batches:   0%|          | 0/15 [00:00<?, ?it/s]

Embeddings computed: torch.Size([951, 768])


### **Hybrid Search Logic** :
Defines the main function that combines ML classification, semantic similarity, and fuzzy matching to find the best answer. (Time: < 5 sec)

In [37]:
X_word_all = word_vect.transform(df['question_proc'])
X_char_all = char_vect.transform(df['question_proc'])
X_all = hstack([X_word_all, X_char_all])

def fuzzy_match_answer(q_proc):
    best = process.extractOne(q_proc, df['question_proc'].tolist(), scorer=fuzz.token_sort_ratio)
    if best and best[1] >= FUZZY_THRESHOLD:
        idx = df.index[df['question_proc'] == best[0]][0]
        return df.loc[idx, 'answer'], best[1]
    return None, 0

def semantic_rerank(q_proc, top_k=5):
    q_emb = embedder.encode(q_proc, convert_to_tensor=True)
    sims = util.cos_sim(q_emb, question_embeddings).cpu().numpy().flatten()
    topk = np.argsort(sims)[-top_k:][::-1]
    best_idx = topk[0]
    return int(best_idx), float(sims[best_idx]), df.iloc[best_idx]['answer']

def answer_hybrid(raw_query, return_debug=False):
    q_proc = normalize_fast(raw_query)
    debug = {'q_proc': q_proc}
    q_vec = hstack([word_vect.transform([q_proc]), char_vect.transform([q_proc])])

    try:
        proba = voting.predict_proba(q_vec)[0]
        nb_best_idx = int(np.argmax(proba))
        nb_best_proba = float(np.max(proba))
        nb_pred_category = le.inverse_transform([nb_best_idx])[0]
    except Exception:
        nb_best_proba = 0.0
        nb_pred_category = None
    debug.update({'nb_proba': nb_best_proba, 'nb_cat': nb_pred_category})

    sem_idx, sem_score, sem_answer = semantic_rerank(q_proc, top_k=5)
    debug.update({'sem_score': sem_score, 'sem_idx': sem_idx})

    fuzz_ans, fuzz_score = fuzzy_match_answer(q_proc)
    debug.update({'fuzz_score': fuzz_score})

    if sem_score >= SEMANTIC_THRESHOLD:
        final = sem_answer
        reason = 'semantic'
    elif fuzz_score >= FUZZY_THRESHOLD:
        final = fuzz_ans
        reason = 'fuzzy'
    elif nb_best_proba >= NB_PROBA_THRESHOLD and nb_pred_category is not None:
        cat_idxs = df.index[df['category'] == nb_pred_category].tolist()
        if cat_idxs:
            cand_vecs = X_all[cat_idxs]
            sims_cand = (q_vec @ cand_vecs.T).toarray().flatten()
            best_local = np.argmax(sims_cand)
            best_global_idx = cat_idxs[best_local]
            final = df.loc[best_global_idx, 'answer']
            reason = 'nb_cat_match'
        else:
            final = sem_answer
            reason = 'nb_no_cat_candidates'
    elif sem_score >= (SEMANTIC_THRESHOLD * 0.8):
        final = sem_answer
        reason = 'semantic_lowconf'
    else:
        final = "Sorry — I don't know the answer to that. Can you rephrase?"
        reason = 'unknown'

    debug.update({'reason': reason})
    return (final, debug) if return_debug else final

# testing
print(answer_hybrid("What are university timings?", return_debug=True))


('University administrative hours are from 8:30 AM to 4:30 PM, Monday to Friday. Classes are scheduled between 8:00 AM to 6:00 PM.', {'q_proc': 'what be university timing', 'nb_proba': 0.8750637618361886, 'nb_cat': 'academic_schedule', 'sem_score': 0.9653139710426331, 'sem_idx': 542, 'fuzz_score': 92.5925925925926, 'reason': 'semantic'})


### **Gradio User Interface** :
Builds and launches the interactive web interface with custom dark mode CSS, connecting the backend logic to the frontend. (Time: Instant) . Start by clicking the link of Gradio and get replies.

In [39]:
import gradio as gr

BG_COLOR = "#0d1117"
CARD_COLOR = "#161b22"
BORDER_COLOR = "#30363d"
ACCENT_COLOR = "#1f6feb"
TEXT_COLOR = "#f0f6fc"

custom_css = f"""
.contain {{
    background-color: {BG_COLOR};
    color: {TEXT_COLOR};
    font-family: 'Segoe UI', system-ui, sans-serif;
    height: 100vh;
    padding: 0;
    margin: 0;
    overflow: hidden;
}}

.news-header {{
    background: {CARD_COLOR};
    border-bottom: 3px solid {ACCENT_COLOR};
    padding: 10px 20px;
    margin: 0;
    position: relative;
}}

.main-title {{
    text-align: center;
    font-size: 28px;
    font-weight: bold;
    color: {ACCENT_COLOR};
    margin: 0;
    padding: 0;
}}

.news-ticker {{
    background: linear-gradient(90deg, {ACCENT_COLOR}, #239b56);
    color: white;
    padding: 8px 0;
    margin: 5px 0 0 0;
    overflow: hidden;
    position: relative;
    font-size: 14px;
    font-weight: 500;
}}

.ticker-content {{
    display: inline-block;
    white-space: nowrap;
    animation: ticker 25s linear infinite;
    padding-left: 100%;
}}

@keyframes ticker {{
    0% {{ transform: translateX(0); }}
    100% {{ transform: translateX(-100%); }}
}}

.ticker-item {{
    display: inline-block;
    padding: 0 30px;
    position: relative;
}}

.ticker-item:after {{
    content: "•";
    position: absolute;
    right: -5px;
    color: white;
}}

.ticker-item:last-child:after {{
    content: "";
}}

.main-content {{
    display: flex;
    flex-direction: column;
    height: calc(100vh - 120px);
    padding: 20px;
    max-width: 900px;
    margin: 0 auto;
}}

.qa-section {{
    flex: 1;
    display: flex;
    flex-direction: column;
    gap: 15px;
    margin-bottom: 20px;
    min-height: 300px;
    max-height: 400px;
}}

.question-box {{
    background: {CARD_COLOR};
    border: 2px solid {ACCENT_COLOR};
    border-radius: 10px;
    padding: 15px;
    margin: 0;
    min-height: 60px;
    max-height: 100px;
    overflow-y: auto;
}}

.question-label {{
    color: {ACCENT_COLOR};
    font-weight: 600;
    font-size: 13px;
    margin-bottom: 5px;
    text-transform: uppercase;
}}

.answer-box {{
    background: {CARD_COLOR};
    border: 2px solid #3fb950;
    border-radius: 10px;
    padding: 15px;
    margin: 0;
    flex: 1;
    min-height: 150px;
    max-height: 250px;
    overflow-y: auto;
}}

.answer-label {{
    color: #3fb950;
    font-weight: 600;
    font-size: 13px;
    margin-bottom: 5px;
    text-transform: uppercase;
}}

.input-section {{
    background: {CARD_COLOR};
    border: 2px solid {BORDER_COLOR};
    border-radius: 10px;
    padding: 15px;
    margin: 0;
}}

.input-row {{
    display: flex;
    gap: 10px;
    align-items: center;
}}

.text-input {{
    background: {BG_COLOR} !important;
    color: {TEXT_COLOR} !important;
    border: 2px solid {BORDER_COLOR} !important;
    border-radius: 8px !important;
    padding: 12px !important;
    font-size: 14px !important;
    flex: 1;
    margin: 0 !important;
}}

.text-input:focus {{
    border-color: {ACCENT_COLOR} !important;
}}

.send-button {{
    background: {ACCENT_COLOR} !important;
    color: white !important;
    border: none !important;
    border-radius: 8px !important;
    padding: 12px 25px !important;
    font-weight: 600 !important;
    height: 44px;
    margin: 0 !important;
}}

.send-button:hover {{
    background: #2a7de9 !important;
}}

.gr-box {{
    border: none !important;
    box-shadow: none !important;
}}

.gr-textbox label {{
    display: none !important;
}}

::-webkit-scrollbar {{
    width: 6px;
}}

::-webkit-scrollbar-track {{
    background: {BG_COLOR};
}}

::-webkit-scrollbar-thumb {{
    background: {BORDER_COLOR};
    border-radius: 3px;
}}

.gr-container, .gr-form, .gr-column {{
    padding: 0 !important;
    margin: 0 !important;
    gap: 0 !important;
}}

footer {{ display: none !important; }}
"""

with gr.Blocks(css=custom_css, theme=gr.themes.Default()) as ui:
    with gr.Column(elem_classes="news-header"):
        gr.Markdown("<div class='main-title'>RIPHAH INTERNATIONAL UNIVERSITY</div>", elem_id="main-title")
        with gr.Column(elem_classes="news-ticker"):
            gr.Markdown("""
                <div class='ticker-content'>
                    <span class='ticker-item'>📞 Contact: +92-51-512-5125</span>
                    <span class='ticker-item'>✉️ Email: info@riphah.edu.pk</span>
                    <span class='ticker-item'>🏛️ 14 Main Campus Locations</span>
                    <span class='ticker-item'>🎓 Excellence in Education Since 2002</span>
                    <span class='ticker-item'>🌍 Islamic Values & Modern Education</span>
                    <span class='ticker-item'>📚 100+ Academic Programs</span>
                    <span class='ticker-item'>👨‍🎓 30,000+ Students</span>
                    <span class='ticker-item'>🏆 HEC Recognized</span>
                </div>
            """, elem_id="news-ticker")

    with gr.Column(elem_classes="main-content"):
        with gr.Column(elem_classes="qa-section"):
            question_display = gr.Textbox(
                label="QUESTION",
                interactive=False,
                elem_classes="question-box",
                show_label=True,
                container=False,
                lines=2,
                max_lines=3,

            )
            answer_display = gr.Textbox(
                label="ANSWER",
                interactive=False,
                elem_classes="answer-box",
                show_label=True,
                container=False,
                lines=6,
                max_lines=8,

            )

        with gr.Column(elem_classes="input-section"):
            with gr.Row(elem_classes="input-row"):
                user_input = gr.Textbox(
                    placeholder="Type your question here and press Enter...",
                    show_label=False,
                    elem_classes="text-input",
                    container=False,
                    scale=4
                )
                send_btn = gr.Button("Get Answer", elem_classes="send-button", scale=1)

    def get_answer(question):
        if not question.strip():
            return "", ""
        answer, _ = answer_hybrid(question, return_debug=True)
        if len(answer) > 1000:
            answer = answer[:1000] + "..."
        return question, answer

    send_btn.click(get_answer, [user_input], [question_display, answer_display]).then(lambda: "", None, [user_input])
    user_input.submit(get_answer, [user_input], [question_display, answer_display]).then(lambda: "", None, [user_input])

ui.launch(share=True, height=600)


/tmp/ipython-input-2026466100.py:210: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Default()) as ui:
/tmp/ipython-input-2026466100.py:210: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Default()) as ui:
/usr/local/lib/python3.12/dist-packages/gradio/components/base.py:200: UserWarning: show_label has no effect when container is False.
  warnings.warn("show_label has no effect when container is False.")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f204e4c2241036af7b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
